# Cvičení 8 - Náhodné lesy, AdaBoost

V dnešním cvičení budeme zkoumat ensemble modely, konkrétně techniky **baggingu** (příkladem bude náhodný les) a **boostingu** (příkladem bude AdaBoost).

Základním principem ensemble modelů je, že k predikci využíváne více "slabých" modelů (tzv. weak learners). Tyto modely jsou buď různé, nebo natrénované na různých datech. Ensemble model potom agreguje predicke těchto modelů a sestaví z nich finální predikci.

Dvě základní ensemble techniky jsou:
* **bagging** - což je zkratka pro **b**ootstrap **agg**regat**ing**. Metoda je založená na trénování totožných podmodelů na náhodných výběrech z datasetu (bootstrapping) s tím, že finální predikce se tvoří jako nejčastější predikovaná hodnota (klasifikace) nebo průměr predikovaných hodnot (regrese). Výsledkem je typicky redukce rozptylu (variance).
* **boosting** - má primárně za cíl redukovat vychýlení (bias). Podmodely stejného typu se konstruují postupně a mají různé váhy. Zároveň dochází k vážení datových bodů tak, aby se pozdější podmodely soustředili především na ty body, které byli těmi předchozími predikovány špatně. 
Finální predikce se vytváří jako vážená nejčastěji predikovaná hodnota (klasifikace) a nebo vážený průměr predikovaných hodnot (klasifikace)

Ensemble modely lze využít jak pro klasifikaci, tak pro regresi! 🚀

In [ ]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, ParameterGrid
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

random_seed = 42

## Dataset 🏘️
Opět využijeme dataset **adult census income** z [Kaggle](https://www.kaggle.com/uciml/adult-census-income).
Cílem úlohy je na základě demografických údajů predikovat, zda příjem člověka přesáhne $50 000 ročně.

Protože, jsme s tímto datasetem již pracovali, pro urychlení přebereme základní předzpracování ze druhého cvičení.

V prvním přiblížení použijeme na nominální příznaky **one-hot encoding**

In [ ]:
def get_adult_census(one_hot = True, fill_na = True):
    df = pd.read_csv('adult-census.csv')
    df = df.replace('?', np.nan)
    df.drop(columns = ["education"], inplace = True)
    income_category = pd.api.types.CategoricalDtype(categories=['<=50K', '>50K'], ordered=True)
    df['income'] = df['income'].astype(income_category)
    sex_category = pd.api.types.CategoricalDtype(categories=['Female', 'Male'])
    df['sex'] = df['sex'].astype(sex_category)
    
    string_cols = ['native.country', 'occupation', 'workclass', 'marital.status', 'relationship', 'race']
    df[string_cols] = df[string_cols].astype('category')
    
    for col in df.select_dtypes('category').columns:
        df[col] = df[col].cat.codes

    # ONE-HOT encoding
    if one_hot:
        df = pd.get_dummies(df)
    # Missing values
    if fill_na:
        df = df.fillna('-1')
    return df

# načteme si dataset
df = get_adult_census()

# Split the training dataset into 60% train and 40% rest
Xtrain, Xrest, ytrain, yrest = train_test_split(df.drop(columns=['income']), df.income, test_size=0.4, random_state=random_seed)

# Split the rest of the data into 0.6*0.4=24% validation, 0.4*0.4=16% test
Xtest, Xval, ytest, yval = train_test_split(Xrest, yrest, test_size=0.6, random_state=random_seed)

print(f"Train rozměry, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val rozměry, X: {Xval.shape}, y: {yval.shape}")
print(f"Test rozměry, X: {Xtest.shape}, y: {ytest.shape}")

Pro porovnání výsledků si nejprve připravíme klasický rozhodovací strom a logistickou regresi.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clfDT = DecisionTreeClassifier(max_depth = 8, random_state = 42)
clfDT.fit(Xtrain, ytrain)
print("Rozhodovací strom")
print('Accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clfDT.predict(Xtrain))))
print('Accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clfDT.predict(Xval))))

from sklearn.linear_model import LogisticRegression
clfLR = LogisticRegression(max_iter = 1000)
clfLR.fit(Xtrain, ytrain)
print("\nLogistická regrese")
print('Accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clfLR.predict(Xtrain))))
print('Accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clfLR.predict(Xval))))

## Bagging - náhodný les

Jako příklad baggingu si zkusíme model náhodného lesa (random forest):
- Trénuje se na bootstrapu dat: vytvoříme **n** množin a do nich vybereme data náhodným výběrem s opakováním.
- Každá množina se použije k natrénování jednoho stromu (vznikne jich tedy n).
- Výstupy ze jednotlivých stromů jsou agregovány do finálního výstupu.
  
<center><img src="img/bagging.png" width="50%"></center>

Použití lesa místo stromu je `sklearn` velice jednoduché:
- stačí nahradit `DecisionTreeClassifier` v kódu výše  třídou `RandomForestClassifier`.
- má skoro stejné parametry jako `DecisionTreeClassifier`. Například
    * `max_depth`, který určuje maximální hloubku jednoho stromu.
    * `max_features`, který určuje počet příznaků, ze kterých si hladový algoritmus vybírá ten, podle kterého bude v aktuálním kroku data "větvit".

Má ale také jeden nový, `n_estimators`, který je zásadní a určuje počet stromů v lese!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier(n_estimators = 20, max_depth = 8, random_state = 42)
clfRF.fit(Xtrain, ytrain)
print("Náhodný les")
print('Accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clfRF.predict(Xtrain))))
print('Accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clfRF.predict(Xval))))

Vidíme, že jsme dostali víceméně stejně výkonný model jako rozhodovací strom výše.

Podívejme se ještě na jeden zajímavý atribut natrénovaného modelu, kterým je `estimators_`. Můžeme tak zkusit ručně zreplikovat provedený bagging.

In [ ]:
# print(clfRF.estimators_)
# napočteme si predikce všech podmodelů
ypred = np.zeros((len(clfRF.estimators_),yval.shape[0]))
for i,e in enumerate(clfRF.estimators_):
    ypred[i,:] = e.predict(np.array(Xval))
    
# Výsledky pro první bod
print("Weak predictions", ypred[:,0])
print("Final", clfRF.predict(Xval)[0])

### Úkol - proveďte ladění hyperparametrů rozhodovacího stromu i náhodného lesa
V obou třídách zkuste najít nejlepší modely a diskutujte výsledky.

In [ ]:
# rozhodovací strom
# Váš kód zde



In [ ]:
# náhodný les
# Váš kód zde



Model náhodného lesa je v tomto případě nepatrně lepší.

### Úkol - vyzkoušejte model náhodného lesa na regresní úloze
Zkuste použít náhodný les na predikci spojité vysvětlované proměnné u datasetu house prices.

Protože, jsme s tímto datasetem již pracovali, pro urychlení přebereme základní předzpracování z třetího cvičení.

In [ ]:
def get_houses_dataset(one_hot = True, fill_na = True):
    df = pd.read_csv('house-prices-train.csv')

    qual_category = pd.api.types.CategoricalDtype(categories=['Po', 'Fa', 'TA', 'Gd', 'Ex'], ordered=True)
    for col in df.select_dtypes('object').columns:
        if col.endswith("Qual") or col.endswith("Qu") or col.endswith("QC") or col.endswith("Cond"):
            df[col] = df[col].astype(qual_category)

    for col in df.select_dtypes('category').columns:
        df[col] = df[col].cat.codes

    # ONE-HOT encoding
    if one_hot:
        df = pd.get_dummies(df)
    # Missing values
    if fill_na:
        df = df.fillna('-1')
    return df

# načteme si dataset
df = get_houses_dataset()

# Split the training dataset into 60% train and 40% rest
Xtrain, Xrest, ytrain, yrest = train_test_split(df.drop(columns=['SalePrice']), df.SalePrice, test_size=0.4, random_state=random_seed)

# Split the rest of the data into 0.6*0.4=24% validation, 0.4*0.4=16% test
Xtest, Xval, ytest, yval = train_test_split(Xrest, yrest, test_size=0.6, random_state=random_seed)

print(f"Train rozměry, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val rozměry, X: {Xval.shape}, y: {yval.shape}")
print(f"Test rozměry, X: {Xtest.shape}, y: {ytest.shape}")

Pro porovnání si zkusíme nejprve rozhodovací strom

In [ ]:
# rozhodovací strom
from sklearn.tree import DecisionTreeRegressor

param_grid = {
    'max_depth': range(1, 15)
}

param_comb = ParameterGrid(param_grid)

val_metric = []
for params in param_comb:
    clf = DecisionTreeRegressor(**params, random_state = 42).fit(Xtrain, ytrain)
    val_metric.append(metrics.mean_squared_error(yval, clf.predict(Xval), squared = False))
    
best_params = param_comb[np.argmin(val_metric)]
print(f"We found the best params {best_params} with validation RMSE {min(val_metric):.0f}.")

Nyní můžete zkusit model `RandomForestRegressor` což je náhodný les pro regresi.

In [ ]:
# Váš kód zde




Vidíme, že náhodný les dosáhl o dost lepšího výsledku.

## Boosting - AdaBoost
Narozdíl od **random forest** používá **AdaBoost** metody boostingu:
- Na datech natrénujeme první strom (stromy se úmyslně volí s malou hloubkou)
- Data ohodnotíme podle "kvality" jejich predikce tímto stromem a přidáme jim váhu (ty co nebyly ohodnoceny dobře mají výšší váhu a opačně)
- Na základě těchto dat natrénujeme další strom, kterým se budeme snažit správně ohodnotit data s vyšší váhou
- Při predikci necháme všechny weak learners predikovat cílovou hodnotu kterou převážíme accuracy jednotlivých stromů a výsledek zagregujeme
    
<center><img src="img/boosting.png" width="50%"></center>

Ve `sklearn` je AdaBoost pro klasifikaci implementován jako `AdaBoostClassifier`

AdaBoostClassifier má dva hlavní volitelné parametry:
- `base_estimator`, který volí co bude použito za estimátor na data, v našem případě tedy **rozhodovací strom**
    - výchozí hodnota je **rozhodovací strom** hloubky 1 (lze využít i jiné modely)
- `n_estimators`, který říká kolik estimátorů (v našem případě **rozhodovacím stromů**) bude vytvořeno
    - výchozí hodnota je 50
    
Dále má parametr `algorithm` jehož hodnota "SAMME" odpovídá přesně tomu, co jsme dělali na přednášce. 

Defaultní hodnota "SAMME.R" potom odpovídá trochu jinému přístupu, kdy jednotlivé podmodely predikují pravděpodobnosti příslušnosti k jednotlivým třídám a při finální predikci se spočítají vážené pravděpodobnosti příslušnosti k jednotlivým třídám a predikuje se třída, která má největší váženou pravděpodobnost.

In [ ]:
# načteme si dataset
df = get_adult_census()

# Split the training dataset into 60% train and 40% rest
Xtrain, Xrest, ytrain, yrest = train_test_split(df.drop(columns=['income']), df.income, test_size=0.4, random_state=random_seed)

# Split the rest of the data into 0.6*0.4=24% validation, 0.4*0.4=16% test
Xtest, Xval, ytest, yval = train_test_split(Xrest, yrest, test_size=0.6, random_state=random_seed)

print(f"Train rozměry, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val rozměry, X: {Xval.shape}, y: {yval.shape}")
print(f"Test rozměry, X: {Xtest.shape}, y: {ytest.shape}")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clfAda = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 5), n_estimators = 10, random_state = 42, algorithm = "SAMME")
clfAda.fit(Xtrain, ytrain)
print("AdaBoost")
print('Accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clfAda.predict(Xtrain))))
print('Accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clfAda.predict(Xval))))

V tuto chvíli jsme nedostali nejlepší model.

Podívejme se ještě zajímavé atributy natrénovaného modelu, kterými jsou `estimators_` a `estimator_weights_`.

In [ ]:
ypred_proba = np.zeros((len(clfAda.estimators_),yval.shape[0],2))
ypred = np.zeros((len(clfAda.estimators_),yval.shape[0]))
for i,e in enumerate(clfAda.estimators_):
    ypred_proba[i,:,:] = e.predict_proba(np.array(Xval))
    ypred[i,:] = e.predict(np.array(Xval))
    
print("Weak predictions\n", ypred[:,0:2])
print("Final", clfAda.predict(Xval)[0:2])

# Protože predikce je vážená, podívejme se na váhy
print("Weights:", clfAda.estimator_weights_)
for i in range(2):
    print(f"Sum pro 0: {np.dot(1-ypred[:,i], clfAda.estimator_weights_):.2f};", f"Sum pro 1: {np.dot(ypred[:,i], clfAda.estimator_weights_):.2f}")

Zopakujme to stejné pro `algorithm = "SAMME.R"` což je defaultní nastavení.

In [ ]:
clfAda = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 5), n_estimators = 10, random_state = 42, algorithm = "SAMME.R")
clfAda.fit(Xtrain, ytrain)
print("AdaBoost")
print('Accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clfAda.predict(Xtrain))))
print('Accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clfAda.predict(Xval))))

Vidíme, že jsme dostali doposud nejlepší model.

Pojďme se podívat na predikce jednotlivých podmodelů.

In [ ]:
ypred_proba = np.zeros((len(clfAda.estimators_),yval.shape[0],2))
ypred = np.zeros((len(clfAda.estimators_),yval.shape[0]))
for i,e in enumerate(clfAda.estimators_):
    ypred_proba[i,:,:] = e.predict_proba(np.array(Xval))
    ypred[i,:] = e.predict(np.array(Xval))
    
print("Weak predictions\n", ypred[:,0:2])
print("Final", clfAda.predict(Xval)[0:2])

# Protože predikce je vážená, podívejme se na váhy
print("Weights:", clfAda.estimator_weights_)
for i in range(2):
    print(f"Sum pro 0: {np.dot(1-ypred[:,i], clfAda.estimator_weights_):.2f};", f"Sum pro 1: {np.dot(ypred[:,i], clfAda.estimator_weights_):.2f}")

Vidíme, že v obou bodech by vážená predikce byla rovna 0. Tak ale při tomto algoritmu model nepostupuje. Ve skutečnosti váží pravděpodobnosti příslušností, které máme v proměnné `ypred_proba`

In [ ]:
print("Weak probability predictions\n", ypred_proba[:,0:2])
print("Final", clfAda.predict(Xval)[0:2])

# Protože predikce je vážená, podívejme se na váhy
print("Weights:", clfAda.estimator_weights_)
for i in range(2):
    print(f"Sum pro 0: {np.dot(ypred_proba[:,i,0], clfAda.estimator_weights_):.2f};", f"Sum pro 1: {np.dot(ypred_proba[:,i,1], clfAda.estimator_weights_):.2f}")

Teď už se naše pozorování shoduje s predikcí modelu.

### Úkol - proveďte ladění hyperparametrů modelu AdaBoost
Diskutujte výsledky.

In [ ]:
# Váš kód zde




Dostali jsme s přehledem nejlepší model

### Úkol - vyzkoušejte model ada boost na regresní úloze
Zkuste použít náhodný les na predikci spojité vysvětlované proměnné u datasetu house prices.

Protože, jsme s tímto datasetem již pracovali, pro urychlení přebereme základní předzpracování z třetího cvičení.

In [ ]:
# načteme si dataset
df = get_houses_dataset()

# Split the training dataset into 60% train and 40% rest
Xtrain, Xrest, ytrain, yrest = train_test_split(df.drop(columns=['SalePrice']), df.SalePrice, test_size=0.4, random_state=random_seed)

# Split the rest of the data into 0.6*0.4=24% validation, 0.4*0.4=16% test
Xtest, Xval, ytest, yval = train_test_split(Xrest, yrest, test_size=0.6, random_state=random_seed)

print(f"Train rozměry, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val rozměry, X: {Xval.shape}, y: {yval.shape}")
print(f"Test rozměry, X: {Xtest.shape}, y: {ytest.shape}")

In [ ]:
# Váš kód zde




Dostali jsme nejlepší model